# Entendendo a estrutura do JSON

In [1]:
import json

In [2]:
with open("collected_tweets_1616067285.357482.txt", "r") as file:
    tweets = file.readlines()

In [3]:
parsed_tweets = [json.loads(json.loads(tweet)) for tweet in tweets]

In [ ]:
parsed_tweets[2]

In [13]:
len(parsed_tweets)

2224

In [42]:
primeiro_tt = parsed_tweets[17]

# Transformando os tweets em uma estrutura relacional

In [6]:
import pandas as pd

In [43]:
df_tratado = pd.DataFrame(primeiro_tt).reset_index(drop=True).iloc[:1]
df_tratado

,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,...,quote_count,reply_count,retweet_count,favorite_count,entities,favorited,retweeted,filter_level,lang,timestamp_ms
0,Thu Mar 18 11:34:49 +0000 2021,1372511806539796487,1372511806539796487,RT @toboso_official: 「DISNEY TWISTED－WONDERLAN...,"<a href=""http://twitter.com/download/iphone"" r...",False,None,None,None,None,...,0,0,0,0,NaN,False,False,low,ja,1616067289372


In [44]:
df_tratado.columns

Index(['created_at', 'id', 'id_str', 'text', 'source', 'truncated',
       'in_reply_to_status_id', 'in_reply_to_status_id_str',
       'in_reply_to_user_id', 'in_reply_to_user_id_str',
       'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place',
       'contributors', 'retweeted_status', 'is_quote_status', 'quote_count',
       'reply_count', 'retweet_count', 'favorite_count', 'entities',
       'favorited', 'retweeted', 'filter_level', 'lang', 'timestamp_ms'],
      dtype='object')

In [45]:
df_tratado.drop(columns=['quote_count', 'reply_count', 'retweet_count', 'favorite_count', 'favorited', 'retweeted', 'user', 'entities'], inplace=True)

In [46]:
df_tratado['user_id'] = primeiro_tt['user']['id']
df_tratado['user_id_str'] = primeiro_tt['user']['id_str']
df_tratado['user_screen_name'] = primeiro_tt['user']['screen_name']
df_tratado['user_location'] = primeiro_tt['user']['location']
df_tratado['user_description'] = primeiro_tt['user']['description']
df_tratado['user_protected'] = primeiro_tt['user']['protected']
df_tratado['user_verified'] = primeiro_tt['user']['verified']
df_tratado['user_followers_count'] = primeiro_tt['user']['followers_count']
df_tratado['user_friends_count'] = primeiro_tt['user']['friends_count']
df_tratado['user_created_at'] = primeiro_tt['user']['created_at']

In [47]:
df_tratado

,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,...,user_id,user_id_str,user_screen_name,user_location,user_description,user_protected,user_verified,user_followers_count,user_friends_count,user_created_at
0,Thu Mar 18 11:34:49 +0000 2021,1372511806539796487,1372511806539796487,RT @toboso_official: 「DISNEY TWISTED－WONDERLAN...,"<a href=""http://twitter.com/download/iphone"" r...",False,None,None,None,None,...,1050631007018266624,1050631007018266624,it0it0it0it,日本,25↑/社畜/夢も腐もなんでもあり/Fate(ギル)/ツイステ(🐚寮・♠️)/鬼滅(風・水・...,False,False,115,293,Fri Oct 12 06:15:26 +0000 2018


In [63]:
user_mentions = []
for mention in primeiro_tt['entities']['user_mentions']:
    base = mention.copy()
    mention.pop('indices', None)
    user_mentions.append(
        pd.DataFrame(mention, index=[0]).rename(columns={
            'screen_name': 'entities_screen_name',
            'name': 'entities_name',
            'id': 'entities_id',
            'id_str': 'entities_id_str'
        })
    )
    
user_mentions

,entities_screen_name,entities_name,entities_id,entities_id_str
0,toboso_official,枢やな_Staff,735347393709912065,735347393709912065


In [64]:
pd.concat(user_mentions, ignore_index=True)

,entities_screen_name,entities_name,entities_id,entities_id_str
0,toboso_official,枢やな_Staff,735347393709912065,735347393709912065


In [66]:
dfs = []
for i in user_mentions:
    dfs.append(
        pd.concat([df_tratado.copy(), i], axis=1)
    )

In [69]:
pd.concat(dfs, ignore_index=True)

,created_at,id,id_str,text,source,truncated,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,...,user_description,user_protected,user_verified,user_followers_count,user_friends_count,user_created_at,entities_screen_name,entities_name,entities_id,entities_id_str
0,Thu Mar 18 11:34:49 +0000 2021,1372511806539796487,1372511806539796487,RT @toboso_official: 「DISNEY TWISTED－WONDERLAN...,"<a href=""http://twitter.com/download/iphone"" r...",False,None,None,None,None,...,25↑/社畜/夢も腐もなんでもあり/Fate(ギル)/ツイステ(🐚寮・♠️)/鬼滅(風・水・...,False,False,115,293,Fri Oct 12 06:15:26 +0000 2018,toboso_official,枢やな_Staff,735347393709912065,735347393709912065


In [7]:
def tweet_para_df(tweet):
    try:
        df_tratado = pd.DataFrame(tweet).reset_index(drop=True).iloc[:1]

        df_tratado.drop(columns=['quote_count', 'reply_count', 'retweet_count', 'favorite_count', 'favorited', 'retweeted', 'user', 'entities'], inplace=True)

        df_tratado['user_id'] = tweet['user']['id']
        df_tratado['user_id_str'] = tweet['user']['id_str']
        df_tratado['user_screen_name'] = tweet['user']['screen_name']
        df_tratado['user_location'] = tweet['user']['location']
        df_tratado['user_description'] = tweet['user']['description']
        df_tratado['user_protected'] = tweet['user']['protected']
        df_tratado['user_verified'] = tweet['user']['verified']
        df_tratado['user_followers_count'] = tweet['user']['followers_count']
        df_tratado['user_friends_count'] = tweet['user']['friends_count']
        df_tratado['user_created_at'] = tweet['user']['created_at']

        user_mentions = []
        for mention in tweet['entities']['user_mentions']:
            base = mention.copy()
            mention.pop('indices', None)
            user_mentions.append(
                pd.DataFrame(mention, index=[0]).rename(columns={
                    'screen_name': 'entities_screen_name',
                    'name': 'entities_name',
                    'id': 'entities_id',
                    'id_str': 'entities_id_str'
                })
            )
        dfs = []
        for i in user_mentions:
            dfs.append(
                pd.concat([df_tratado.copy(), i], axis=1)
            )
        df_final = pd.concat(dfs, ignore_index=True) if len(dfs) > 0 else df_tratado
    except:
        return None
    return df_final

In [8]:
%%time
parsed = [tweet_para_df(tweet) for tweet in parsed_tweets]
print(len(parsed))

2224
CPU times: user 27 s, sys: 70.6 ms, total: 27 s
Wall time: 27.3 s


In [9]:
parsed = [i for i in parsed if i is not None]

In [10]:
len(parsed)

1684

In [18]:
tweets_as_df: pd.DataFrame = pd.concat(parsed, ignore_index=True)

In [ ]:
tweets_as_df

# Twitter data ingestion into SQL Sever

In [15]:
import pyodbc
import sqlalchemy

In [22]:
engine = sqlalchemy.create_engine("mssql+pyodbc://SA:P@ssw0rd@localhost/twitter?driver=ODBC+Driver+17+for+SQL+Server")

In [24]:
# Ingestion
tweets_as_df.to_sql("tweets", con=engine, index=False, if_exists="append")